# Evaluation BERT, previously fine-tuned on MultiNLI datsaset, on Figurative Language

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
! pip install datasets
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [18]:
import torch
import pandas as pd
import numpy as np
import os
import datetime

from transformers import (BertTokenizer,
                          AutoModelForSequenceClassification,
                          Trainer,
                          TrainingArguments,
                          TrainerCallback)
from datasets import (Dataset,
                      load_dataset,
                      load_metric,
                      concatenate_datasets,
                      ClassLabel,
                      Value,
                      Features)

In [4]:
BATCH_SIZE = 32

In [5]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)


cuda:0


## Model

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], padding="max_length", truncation=True)

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("an-eve/bert-base-uncased-mnli-2-labels")
model.to(DEVICE)
model.eval()

config.json:   0%|          | 0.00/780 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [9]:
metric = load_metric('glue', "mnli")

<ipython-input-9-17bf5bf84e68>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', "mnli")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

## IMPLI (idioms)

In [12]:
url_ne = "https://github.com/an-eve/nlp-nli-idioms/raw/main/dataset/full_ne.tsv"
url_e =  "https://github.com/an-eve/nlp-nli-idioms/raw/main/dataset/full_e.tsv"

In [13]:
full_ne = load_dataset('csv', data_files=url_ne, delimiter='\t', column_names = ['premise', 'hypothesis', 'label'], split='train')
full_e = load_dataset('csv', data_files=url_e, delimiter='\t', column_names = ['premise', 'hypothesis', 'label'], split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
def modify_label_ne(example):
    example['label'] = 1
    return example

def modify_label_e(example):
    example['label'] = 0
    return example

full_ne = full_ne.map(modify_label_ne)
full_e = full_e.map(modify_label_e)

Map:   0%|          | 0/7547 [00:00<?, ? examples/s]

Map:   0%|          | 0/16172 [00:00<?, ? examples/s]

In [15]:
new_features = full_ne.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

full_ne = full_ne.cast(new_features)
full_e = full_e.cast(new_features)

Casting the dataset:   0%|          | 0/7547 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/16172 [00:00<?, ? examples/s]

In [16]:
print(full_ne)
print(full_e)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 7547
})
Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 16172
})


In [17]:
print(full_ne[:2], '\n')
print(full_e[:2])

{'premise': ['13 , 1991, in the fourth week of the U.S.-led air war against Iraq, a British Tornado warplane dropped a bomb that was intended to take out a key river bridge at Fallujah.', "16) of the evacuation of Attica that the Athenians took it so badly because it was like leaving one's polis; this is on the face of it a paradox because they were going from their country demes to the polis."], 'hypothesis': ['13 , 1991, in the fourth week of the U.S.-led air war against Iraq, a British Tornado warplane gave decisive news that was intended to take out a key river bridge at Fallujah.', "16) of the evacuation of Attica that the Athenians took it so badly because it was like leaving one's polis; this is Encountering a paradox because they were going from their country demes to the polis."], 'label': [1, 1]} 

{'premise': ['10/10 SUPERB.í‰äó\x9d David Jemitus, Surrey, June 2006 í‰äóìAfter studying Reiki 1 with a Western-style Master, Reiki Evolution has been a breath of fresh air.', "( 1

In [19]:
full = concatenate_datasets([full_ne, full_e])
full = full.shuffle(seed=128)

In [20]:
print(full, '\n')

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 23719
}) 



In [21]:
full.push_to_hub("an-eve/impli", private=True)
full_ne.push_to_hub("an-eve/impli_non_entailment", private=True)
full_e.push_to_hub("an-eve/impli_entailment", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/24 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/impli_entailment/commit/ec8c131eb40a57e1269e5dc792f66f34a1617a02', commit_message='Upload dataset', commit_description='', oid='ec8c131eb40a57e1269e5dc792f66f34a1617a02', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
tokenized_full = full.map(tokenize_function, batched=True)
tokenized_full_ne = full_ne.map(tokenize_function, batched=True)
tokenized_full_e = full_e.map(tokenize_function, batched=True)

Map:   0%|          | 0/23719 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/7547 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/16172 [00:00<?, ? examples/s]

In [23]:
args_test = TrainingArguments("IMPLI",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_full,
    eval_dataset=tokenized_full,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.4700125455856323,
 'eval_accuracy': 0.6369155529322484,
 'eval_runtime': 847.0282,
 'eval_samples_per_second': 28.003,
 'eval_steps_per_second': 0.876}

In [25]:
args_test = TrainingArguments("IMPLI-ne",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_full_ne,
    eval_dataset=tokenized_full_ne,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 2.235067367553711,
 'eval_accuracy': 0.4436199814495826,
 'eval_runtime': 271.3389,
 'eval_samples_per_second': 27.814,
 'eval_steps_per_second': 0.87}

In [24]:
args_test = TrainingArguments("IMPLI-e",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_full_e,
    eval_dataset=tokenized_full_e,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.1129837036132812,
 'eval_accuracy': 0.72712094978976,
 'eval_runtime': 580.6845,
 'eval_samples_per_second': 27.85,
 'eval_steps_per_second': 0.871}

## Idioms

### Adversarial Definitions
Silver non-entailment, replacing an idiom used in a figuartive context with adversarial definition

In [ ]:
url_magpie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/adversarial_definition_ne_magpie.tsv"
url_pie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/adversarial_definition_ne_pie.tsv"
url_semeval = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/adversarial_definition_ne_semeval.tsv"

In [ ]:
adv_magpie_df = pd.read_csv(url_magpie, sep='\t', names=['premise','hypothesis', 'label'])
adv_pie_df = pd.read_csv(url_pie, sep='\t', names=['premise','hypothesis', 'label'])
adv_semeval_df = pd.read_csv(url_semeval, sep='\t', names=['premise','hypothesis', 'label'])

In [ ]:
adv_magpie_df['label'] = 1
adv_pie_df['label'] = 1
adv_semeval_df['label'] = 1

In [ ]:
adv_ne_df = pd.concat([adv_magpie_df, adv_pie_df, adv_semeval_df], ignore_index=True)
print("Adversarial Definitions Dataframe shape: ", adv_ne_df.shape)
adv_ne_df.head()

Adversarial Definitions Dataframe shape:  (6116, 3)


,premise,hypothesis,label
0,"Taking the EG-1 first, the HFS bridge humbucke...","Taking the EG-1 first, the HFS bridge humbucke...",1
1,At the OK Corral — as she preferred to call wh...,At the OK Corral — as she preferred to call wh...,1
2,That is how he ducks out of his responsibiliti...,That is how he ducks out of his responsibiliti...,1
3,"Without his moustache, the little spin magicia...","Without his moustache, the little spin magicia...",1
4,"Apartheid arguments over, the Commonwealth's n...","Apartheid arguments over, the Commonwealth's n...",1


In [ ]:
adv_ne_df.drop_duplicates(inplace=True, ignore_index=True)
print("Adversarial Definitions Dataframe shape after considering duplicates: ", adv_ne_df.shape)

Adversarial Definitions Dataframe shape after considering duplicates:  (6035, 3)


In [ ]:
adv_ne_dataset = Dataset.from_pandas(adv_ne_df)
print(adv_ne_dataset)
print(adv_ne_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 6035
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [ ]:
new_features = adv_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

adv_ne_dataset = adv_ne_dataset.cast(new_features)

print(adv_ne_dataset.features)

Casting the dataset:   0%|          | 0/6035 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
adv_ne_dataset[:10]

{'premise': ["Taking the EG-1 first, the HFS bridge humbucker delivers a warm, fat sound reminiscent of a vintage PAF, and while the guitar doesn't have the image of a rocker the bridge pickup is almost arrogant in the way it handles an amp at full tilt.",
  "At the OK Corral — as she preferred to call what some of its visitors termed her quim (the phrase struck her as useful after she'd seen the Western)— some men proved themselves and others came to grief, and both kinds had male competitors on their mind and wanted to outgun them.",
  'That is how he ducks out of his responsibilities and will not come clean — Now I shall sit down.',
  'Without his moustache, the little spin magician looked more boyish than ever, prompting the thought that unless he cooled down he might benefit from six of the best from the carpet - slipper.',
  "Apartheid arguments over, the Commonwealth's now seeking a new role spreading democracy, but the assembly lacks stars and so far the Cypriots have stolen th

In [ ]:
adv_ne_dataset.push_to_hub("an-eve/adversarial-non-entailment", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/adversarial-non-entailment/commit/c6a88e000ee81022d3a22f2ec0f1db3d15d776fb', commit_message='Upload dataset', commit_description='', oid='c6a88e000ee81022d3a22f2ec0f1db3d15d776fb', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_adv_ne_dataset = adv_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6035 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
args_test = TrainingArguments("Adversarial-Definitions",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_adv_ne_dataset,
    eval_dataset=tokenized_adv_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 2.4292047023773193,
 'eval_accuracy': 0.40314830157415077,
 'eval_runtime': 205.4416,
 'eval_samples_per_second': 29.376,
 'eval_steps_per_second': 0.92}

### Figurative Context
Silver entailment, replacing an idiom used in a figuartive context with definition

In [ ]:
url_magpie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/fig_context_magpie_e.tsv"
url_pie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/fig_context_pie_e.tsv"
url_semeval = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/fig_context_semeval_e.tsv"

In [ ]:
fig_magpie_df = pd.read_csv(url_magpie, sep='\t', names=['premise','hypothesis', 'label'])
fig_pie_df = pd.read_csv(url_pie, sep='\t', names=['premise','hypothesis', 'label'])
fig_semeval_df = pd.read_csv(url_semeval, sep='\t', names=['premise','hypothesis', 'label'])

In [ ]:
fig_magpie_df['label'] = 0
fig_pie_df['label'] = 0
fig_semeval_df['label'] = 0

In [ ]:
fig_e_df = pd.concat([fig_magpie_df, fig_pie_df, fig_semeval_df], ignore_index=True)
print("Figurative Context Dataframe shape: ", fig_e_df.shape)
fig_e_df.head()

Figurative Context Dataframe shape:  (16652, 3)


,premise,hypothesis,label
0,"‘ Well, it's a recognised symptom of the condi...","‘ Well, it's a recognised symptom of the condi...",0
1,"‘ Well, it's a recognised symptom of the condi...","‘ Well, it's a recognised symptom of the condi...",0
2,"Taking the EG-1 first, the HFS bridge humbucke...","Taking the EG-1 first, the HFS bridge humbucke...",0
3,The Prime Minister believes that her words str...,The Prime Minister believes that her words eli...,0
4,"He was dressed more casually, and, just as he ...","He was dressed more casually, and, just as he ...",0


In [ ]:
fig_e_df.drop_duplicates(inplace=True, ignore_index=True)
print("Figurative Contex Dataframe shape after considering duplicates: ", fig_e_df.shape)

Figurative Contex Dataframe shape after considering duplicates:  (15641, 3)


In [ ]:
fig_e_dataset = Dataset.from_pandas(fig_e_df)
print(fig_e_dataset)
print(fig_e_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 15641
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [ ]:
new_features = fig_e_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

fig_e_dataset = fig_e_dataset.cast(new_features)

print(fig_e_dataset.features)

Casting the dataset:   0%|          | 0/15641 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
fig_e_dataset[:10]

{'premise': ["‘ Well, it's a recognised symptom of the condition that an attack can come out of the blue.",
  "‘ Well, it's a recognised symptom of the condition that an attack can come out of the blue.",
  "Taking the EG-1 first, the HFS bridge humbucker delivers a warm, fat sound reminiscent of a vintage PAF, and while the guitar doesn't have the image of a rocker the bridge pickup is almost arrogant in the way it handles an amp at full tilt.",
  'The Prime Minister believes that her words strike a chord with the British public.',
  'He was dressed more casually, and, just as he looked good — and then some — in a business suit, Leith realised that to some women his tall, casually clad self was quite something more.',
  'That is how he ducks out of his responsibilities and will not come clean — Now I shall sit down.',
  'That is how he ducks out of his responsibilities and will not come clean — Now I shall sit down.',
  'STRAIGHT FROM THE SHOULDER',
  'He too (‘ birds of a feather’) h

In [ ]:
fig_e_dataset.push_to_hub("an-eve/figurative-entailment", private=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/figurative-entailment/commit/77e8d6422cf98c71de84ef32b9b951a82696a5a3', commit_message='Upload dataset', commit_description='', oid='77e8d6422cf98c71de84ef32b9b951a82696a5a3', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_fig_e_dataset = fig_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/15641 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Figurative-Context",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_fig_e_dataset,
    eval_dataset=tokenized_fig_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.1207855939865112,
 'eval_accuracy': 0.7255290582443578,
 'eval_runtime': 533.8898,
 'eval_samples_per_second': 29.296,
 'eval_steps_per_second': 0.916}

### Literal Context
Silver non-entailment, replacing an idiom used in a literal context with definition

In [ ]:
url_magpie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/lit_context_magpie_ne.tsv"
url_pie = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/lit_context_pie_ne.tsv"
url_semeval = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/lit_context_semeval_ne.tsv"

In [ ]:
lit_magpie_df = pd.read_csv(url_magpie, sep='\t', names=['premise','hypothesis', 'label'])
lit_pie_df = pd.read_csv(url_pie, sep='\t', names=['premise','hypothesis', 'label'])
lit_semeval_df = pd.read_csv(url_semeval, sep='\t', names=['premise','hypothesis', 'label'])

In [ ]:
lit_magpie_df['label'] = 1
lit_pie_df['label'] = 1
lit_semeval_df['label'] = 1

In [ ]:
lit_ne_df = pd.concat([lit_magpie_df, lit_pie_df, lit_semeval_df], ignore_index=True)
print("Literal Context Dataframe shape: ", lit_ne_df.shape)
lit_ne_df.head()

Literal Context Dataframe shape:  (886, 3)


,premise,hypothesis,label
0,"One night of winter, with howling winds blowin...","One night of winter, with howling winds blowin...",1
1,Murder in the docks.,Murder under scrutiny.,1
2,Murder in the docks.,Murder under close scrutiny.,1
3,Nobody gets away with murder in this City.’,Nobody does bad things and not be punished in ...,1
4,Nicolaus speculates that this could give ecolo...,Nicolaus speculates that this could give ecolo...,1


In [ ]:
lit_ne_df.drop_duplicates(inplace=True, ignore_index=True)
print("Literal Contex Dataframe shape after considering duplicates: ", lit_ne_df.shape)

Literal Contex Dataframe shape after considering duplicates:  (881, 3)


In [ ]:
lit_ne_dataset = Dataset.from_pandas(lit_ne_df)
print(lit_ne_dataset)
print(lit_ne_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 881
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [ ]:
new_features = lit_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

lit_ne_dataset = lit_ne_dataset.cast(new_features)

print(lit_ne_dataset.features)

Casting the dataset:   0%|          | 0/881 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
lit_ne_dataset[:10]

{'premise': ['One night of winter, with howling winds blowing and heavy rain lashing the streets, he arrived cold and wet at the Club, went straight to the bar as usual, and was given his usual drink by the bar man, ‘ Johnny’.',
  'Murder in the docks.',
  'Murder in the docks.',
  'Nobody gets away with murder in this City.’',
  'Nicolaus speculates that this could give ecologists a method of controlling predation on eggs of rare birds.',
  'Vehicles for off- and on - road use feature in our special Country Car features.',
  'Half a minute later, as he turned the corner into Oxford Street, he was blown off his feet by the explosion just yards from his cab.',
  'He turned the corner miserably, bumping into Elsie with their fish and chips.',
  "These less direct consequences of man's animality could, Malinowski argued, be understood in the light of their integrative contribution to man's social existence.",
  'Switzerland is famous for six cheeses, four of which belong to the semi - har

In [ ]:
lit_ne_dataset.push_to_hub("an-eve/literal-non-entailment", private=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/literal-non-entailment/commit/5bdb65287f16a69890533105f400b2aff85826c1', commit_message='Upload dataset', commit_description='', oid='5bdb65287f16a69890533105f400b2aff85826c1', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_lit_ne_dataset = lit_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/881 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Literal-Context",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_lit_ne_dataset,
    eval_dataset=tokenized_lit_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.1655007600784302,
 'eval_accuracy': 0.6787741203178207,
 'eval_runtime': 31.4182,
 'eval_samples_per_second': 28.041,
 'eval_steps_per_second': 0.891}

### Antonyms

Gold non-entailment, manual replacement of idiomatic definition with antonym

In [ ]:
url_ant = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/manual_antonyms_ne.tsv"

In [ ]:
ant_df = pd.read_csv(url_ant, sep='\t', names=['premise','hypothesis', 'label'])

In [ ]:
ant_df['label'] = 1

In [ ]:
print("Antonyms Dataframe shape: ", ant_df.shape)
ant_df.head()

Antonyms Dataframe shape:  (375, 3)


,premise,hypothesis,label
0,How have you weathered the storm ?,How have you failed in getting through the dif...,1
1,He prefers acting with other countries to goin...,He prefers acting with other countries to doin...,1
2,The imagination trembles at some of these idea...,The imagination trembles at some of these idea...,1
3,"Attractive , popular and smart , Veronica hove...","Attractive , popular and smart , Veronica hove...",1
4,"All her insecurities surged painfully as , thr...","All her insecurities surged painfully as , car...",1


In [ ]:
ant_ne_dataset = Dataset.from_pandas(ant_df)
print(ant_ne_dataset)
print(ant_ne_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 375
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [ ]:
new_features = ant_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

ant_ne_dataset = ant_ne_dataset.cast(new_features)

print(ant_ne_dataset.features)

Casting the dataset:   0%|          | 0/375 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
ant_ne_dataset[:10]

{'premise': ['How have you weathered the storm ?',
  'He prefers acting with other countries to going it alone .',
  "The imagination trembles at some of these ideas — will a profession 's ruling body really come clean about its members ' income?— but no suggestion , on this subject , should be dismissed out of hand .",
  'Attractive , popular and smart , Veronica hovers in the orbit of the ‘ Heathers’ , a four - girl clique who rule the roost at Westerburgh High .',
  'All her insecurities surged painfully as , throwing caution to the winds , she confronted him furiously , demanding imperiously , ‘ Why exactly have you brought me here , Rune ?',
  "It was pretty interesting, like one of the guys goes native and one of the guys doesn't.",
  '‘ It breaks my heart that his career has been ruined .',
  'For me it is quite out of the question that the Maastricht treaties should be changed again in one way or another .',
  "London is like ‘ iced water , it 's crystal clear and safe’",
  'Yo

In [ ]:
ant_ne_dataset.push_to_hub("an-eve/antonym-non-entailment", private=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/antonym-non-entailment/commit/372ef3b4da001561b1474c19016a29ae13791112', commit_message='Upload dataset', commit_description='', oid='372ef3b4da001561b1474c19016a29ae13791112', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_ant_ne_dataset = ant_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/375 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Antonym",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_ant_ne_dataset,
    eval_dataset=tokenized_ant_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 1.255462646484375,
 'eval_accuracy': 0.6533333333333333,
 'eval_runtime': 13.2846,
 'eval_samples_per_second': 28.228,
 'eval_steps_per_second': 0.903}

### Definition
Gold entailment, hand written literal definition of idiomatic sentence

In [ ]:
url_def = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/manual_e.tsv"

In [ ]:
def_df = pd.read_csv(url_def, sep='\t', names=['premise','hypothesis', 'label'])

In [ ]:
def_df['label'] = 0

In [ ]:
print("Definition Dataframe shape: ", def_df.shape)
def_df.head()

Definition Dataframe shape:  (528, 3)


,premise,hypothesis,label
0,How have you weathered the storm ?,How have you succeeded in getting through the ...,0
1,"And then , the Daily Telegraph discovered ‘ th...","And then , the Daily Telegraph discovered ‘ th...",0
2,He prefers acting with other countries to goin...,He prefers acting with other countries to doin...,0
3,The imagination trembles at some of these idea...,The imagination trembles at some of these idea...,0
4,"Attractive , popular and smart , Veronica hove...","Attractive , popular and smart , Veronica hove...",0


In [ ]:
def_e_dataset = Dataset.from_pandas(def_df)
print(def_e_dataset)
print(def_e_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 528
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [ ]:
new_features = def_e_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

def_e_dataset = def_e_dataset.cast(new_features)

print(def_e_dataset.features)

Casting the dataset:   0%|          | 0/528 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
def_e_dataset[:10]

{'premise': ['How have you weathered the storm ?',
  "And then , the Daily Telegraph discovered ‘ the truth’ : ‘ privatisation of Britain 's water industry … runs against the grain of Mr Delors’ social strategy.’",
  'He prefers acting with other countries to going it alone .',
  "The imagination trembles at some of these ideas — will a profession 's ruling body really come clean about its members ' income?— but no suggestion , on this subject , should be dismissed out of hand .",
  'Attractive , popular and smart , Veronica hovers in the orbit of the ‘ Heathers’ , a four - girl clique who rule the roost at Westerburgh High .',
  'All her insecurities surged painfully as , throwing caution to the winds , she confronted him furiously , demanding imperiously , ‘ Why exactly have you brought me here , Rune ?',
  "It was pretty interesting, like one of the guys goes native and one of the guys doesn't.",
  '‘ It breaks my heart that his career has been ruined .',
  'For me it is quite out o

In [ ]:
def_e_dataset.push_to_hub("an-eve/definition-entailment", private=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/definition-entailment/commit/131feb5d324e9678d2801888624249ce4a85ae59', commit_message='Upload dataset', commit_description='', oid='131feb5d324e9678d2801888624249ce4a85ae59', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_def_e_dataset = def_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/528 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Definition",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_def_e_dataset,
    eval_dataset=tokenized_def_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.8807476162910461,
 'eval_accuracy': 0.7746212121212122,
 'eval_runtime': 18.3317,
 'eval_samples_per_second': 28.803,
 'eval_steps_per_second': 0.927}

### Non-entailment Hand
Gold non-entailment, hand written non-entailed sentence from idiom

In [ ]:
url_hand = "https://github.com/UKPLab/acl2022-impli/raw/main/idioms/manual_ne.tsv"

In [ ]:
hand_df = pd.read_csv(url_hand, sep='\t', names=['premise','hypothesis', 'label'])

In [ ]:
hand_df['label'] = 1

In [ ]:
print("Non-entailment Hand Dataframe shape: ", hand_df.shape)
hand_df.head()

Non-entailment Hand Dataframe shape:  (254, 3)


,premise,hypothesis,label
0,How have you weathered the storm ?,How have you calmed the storm?,1
1,"And then , the Daily Telegraph discovered ‘ th...","And then , the Daily Telegraph discovered ‘ th...",1
2,"All her insecurities surged painfully as , thr...","All her insecurities surged painfully as , thr...",1
3,"It was pretty interesting, like one of the guy...","It was pretty interesting, like one of the guy...",1
4,‘ It breaks my heart that his career has been ...,‘ It touched my heart that his career has been...,1


In [ ]:
hand_ne_dataset = Dataset.from_pandas(hand_df)
print(hand_ne_dataset)
print(hand_ne_dataset.features)

In [ ]:
new_features = hand_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

hand_ne_dataset = hand_ne_dataset.cast(new_features)

print(hand_ne_dataset.features)

Casting the dataset:   0%|          | 0/254 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
hand_ne_dataset[:10]

{'premise': ['How have you weathered the storm ?',
  "And then , the Daily Telegraph discovered ‘ the truth’ : ‘ privatisation of Britain 's water industry … runs against the grain of Mr Delors’ social strategy.’",
  'All her insecurities surged painfully as , throwing caution to the winds , she confronted him furiously , demanding imperiously , ‘ Why exactly have you brought me here , Rune ?',
  "It was pretty interesting, like one of the guys goes native and one of the guys doesn't.",
  '‘ It breaks my heart that his career has been ruined .',
  'For me it is quite out of the question that the Maastricht treaties should be changed again in one way or another .',
  'Only elves and trolls had survived the coming of Man to the discworld ; the elves because they were altogether too clever by half , and the trollen folk because they were at least as good as humans at being nasty , spiteful and greedy .',
  'If the front rabbit sits tight and declines to budge there is no way the others ca

In [ ]:
hand_ne_dataset.push_to_hub("an-eve/hand-non-entailment", private=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/hand-non-entailment/commit/bf82a502714df2f53586a56c60e41ffe2947667c', commit_message='Upload dataset', commit_description='', oid='bf82a502714df2f53586a56c60e41ffe2947667c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_hand_ne_dataset = hand_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/254 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Hand N-E",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_hand_ne_dataset,
    eval_dataset=tokenized_hand_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 2.7674906253814697,
 'eval_accuracy': 0.28346456692913385,
 'eval_runtime': 8.6259,
 'eval_samples_per_second': 29.446,
 'eval_steps_per_second': 0.927}

## Metaphors

### Paraphrase
Gold entailment, hand written literal paraphrase of metaphors

In [ ]:
url_man_met = "https://github.com/UKPLab/acl2022-impli/raw/main/metaphors/manual_e.tsv"

In [ ]:
man_met_df = pd.read_csv(url_man_met, sep='\t', names=['premise','hypothesis', 'label'])

In [ ]:
man_met_df['label'] = 0

In [ ]:
print("Metaphors Paraphrase Dataframe shape: ", man_met_df.shape)
man_met_df.head()

Metaphors Paraphrase Dataframe shape:  (387, 3)


,premise,hypothesis,label
0,he absorbed the knowledge or beliefs of his tr...,he mentally assimilated the knowledge or belie...,0
1,he absorbed the costs for the accident.,he paid for the costs for the accident .,0
2,the sales tax is absorbed into the state incom...,the sales tax is incorporated into the state i...,0
3,the immigrants were quickly absorbed into soci...,the immigrants were quickly assimilated into s...,0
4,her interest in butterflies absorbs her comple...,her interest in butterflies engages her comple...,0


In [ ]:
man_met_e_dataset = Dataset.from_pandas(man_met_df)
print(man_met_e_dataset)
print(man_met_e_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 387
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [ ]:
new_features = man_met_e_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

man_met_e_dataset = man_met_e_dataset.cast(new_features)

print(man_met_e_dataset.features)

Casting the dataset:   0%|          | 0/387 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
man_met_e_dataset[:10]

{'premise': ['he absorbed the knowledge or beliefs of his tribe.',
  'he absorbed the costs for the accident.',
  'the sales tax is absorbed into the state income tax.',
  'the immigrants were quickly absorbed into society.',
  'her interest in butterflies absorbs her completely.',
  'you must adhere to the rules.',
  'the residents of this village adhered to catholicism.',
  'this problem admits of no solution',
  'she aired her opinions on welfare',
  'this idea will never amount to anything'],
 'hypothesis': ['he mentally assimilated the knowledge or beliefs of his tribe .',
  'he paid for the costs for the accident .',
  'the sales tax is incorporated into the state income tax .',
  'the immigrants were quickly assimilated into society .',
  'her interest in butterflies engages her completely .',
  'you must obey the rules .',
  'the residents of this village were devoted in belief to catholicism .',
  'this problem allows for no solution',
  'she openly discussed her opinions on w

In [ ]:
man_met_e_dataset.push_to_hub("an-eve/man-metaphors-entailment", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/man-metaphors-entailment/commit/d56a48e303c8efe671dfb86b7e42ee1c86e3033d', commit_message='Upload dataset', commit_description='', oid='d56a48e303c8efe671dfb86b7e42ee1c86e3033d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_man_met_e_dataset = man_met_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/387 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Metaphors Paraphrase",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_man_met_e_dataset,
    eval_dataset=tokenized_man_met_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.7740918397903442,
 'eval_accuracy': 0.8217054263565892,
 'eval_runtime': 14.2162,
 'eval_samples_per_second': 27.222,
 'eval_steps_per_second': 0.914}

### Non-emtailment Manual
Gold non-entailment, hand written non-entailment of metaphors

In [ ]:
url_ne_met = "https://github.com/UKPLab/acl2022-impli/raw/main/metaphors/manual_ne.tsv"

In [ ]:
ne_met_df = pd.read_csv(url_ne_met, sep='\t', names=['premise','hypothesis', 'label'])

In [ ]:
ne_met_df['label'] = 1

In [ ]:
print("N-E Paraphrase Dataframe shape: ", ne_met_df.shape)
ne_met_df.head()

N-E Paraphrase Dataframe shape:  (281, 3)


,premise,hypothesis,label
0,he absorbed the knowledge or beliefs of his tr...,he absorbed the beverages of his tribe .,1
1,he absorbed the costs for the accident.,he absorbed the sunlight after the accident .,1
2,the sales tax is absorbed into the state incom...,the dirty water is absorbed into the clean wat...,1
3,the immigrants were quickly absorbed into soci...,the chemicals were quickly absorbed into spong...,1
4,her interest in butterflies absorbs her comple...,her butterflies absorb her food completely .,1


In [ ]:
met_ne_dataset = Dataset.from_pandas(ne_met_df)
print(met_ne_dataset)
print(met_ne_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 281
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [ ]:
new_features = met_ne_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

met_ne_dataset = met_ne_dataset.cast(new_features)

print(met_ne_dataset.features)

Casting the dataset:   0%|          | 0/281 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
met_ne_dataset[:10]

{'premise': ['he absorbed the knowledge or beliefs of his tribe.',
  'he absorbed the costs for the accident.',
  'the sales tax is absorbed into the state income tax.',
  'the immigrants were quickly absorbed into society.',
  'her interest in butterflies absorbs her completely.',
  'you must adhere to the rules.',
  'the residents of this village adhered to catholicism.',
  'this problem admits of no solution',
  'she aired her opinions on welfare',
  'this idea will never amount to anything'],
 'hypothesis': ['he absorbed the beverages of his tribe .',
  'he absorbed the sunlight after the accident .',
  'the dirty water is absorbed into the clean water .',
  'the chemicals were quickly absorbed into sponges .',
  'her butterflies absorb her food completely .',
  'you must adhere the rules to the wall .',
  'the wax of this village adhered to fingers .',
  'this park admits no dogs',
  'she aired her laundry on the roof',
  'this budget will never amount to more than a thousand euro

In [ ]:
met_ne_dataset.push_to_hub("an-eve/metaphors-non-entailment", private=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/metaphors-non-entailment/commit/260acf4b1327add7ca8c063107b2536704f92aa0', commit_message='Upload dataset', commit_description='', oid='260acf4b1327add7ca8c063107b2536704f92aa0', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_met_ne_dataset = met_ne_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/281 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("N-E Metaphors Paraphrase",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=42)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_met_ne_dataset,
    eval_dataset=tokenized_met_ne_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.5359946489334106,
 'eval_accuracy': 0.8576512455516014,
 'eval_runtime': 9.9273,
 'eval_samples_per_second': 28.306,
 'eval_steps_per_second': 0.907}

### Automatic Metaphor
Silver entailment, automatic replacement of metaphoric expressions with literal

In [ ]:
url_cc = "https://github.com/UKPLab/acl2022-impli/raw/main/metaphors/replacement_cc_e.tsv"
url_tsvetkov = "https://github.com/UKPLab/acl2022-impli/raw/main/metaphors/replacement_tsvetkov_e.tsv"

In [ ]:
cc_df = pd.read_csv(url_cc, sep='\t', names=['premise','hypothesis', 'label'])
tsvetkov_df = pd.read_csv(url_tsvetkov, sep='\t', names=['premise','hypothesis', 'label'], encoding='unicode_escape')

In [ ]:
cc_df['label'] = 0
tsvetkov_df['label'] = 0

In [ ]:
aut_e_df = pd.concat([cc_df, tsvetkov_df], ignore_index=True)
print("Automatic Metaphors Dataframe shape: ", aut_e_df.shape)
aut_e_df.head()

Automatic Metaphors Dataframe shape:  (645, 3)


,premise,hypothesis,label
0,Get new listings as they hit the market,Get new listings as they became available for ...,0
1,Salt therapy as practiced at The Salt Room Lak...,Salt therapy as practiced at The Salt Room Lak...,0
2,When your primary insurance falls short of cov...,When your primary insurance falls short of pay...,0
3,Please arrive at least 15 minutes early if you...,Please arrive at least 15 minutes early if you...,0
4,Rubberized TPU around the toe box for both med...,Rubberized TPU around the toe box for both med...,0


In [ ]:
aut_e_df.drop_duplicates(inplace=True, ignore_index=True)
print("Automatic Metaphors Dataframe shape after considering duplicates: ", aut_e_df.shape)

Automatic Metaphors Dataframe shape after considering duplicates:  (644, 3)


In [ ]:
aut_e_dataset = Dataset.from_pandas(aut_e_df)
print(aut_e_dataset)
print(aut_e_dataset.features)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 644
})
{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': Value(dtype='int64', id=None)}


In [ ]:
new_features = aut_e_dataset.features.copy()
new_features['label'] = ClassLabel(num_classes = 2, names=["entailment", "non-entailment"])

aut_e_dataset = aut_e_dataset.cast(new_features)

print(aut_e_dataset.features)

Casting the dataset:   0%|          | 0/644 [00:00<?, ? examples/s]

{'premise': Value(dtype='string', id=None), 'hypothesis': Value(dtype='string', id=None), 'label': ClassLabel(names=['entailment', 'non-entailment'], id=None)}


In [ ]:
aut_e_dataset[:10]

{'premise': ['Get new listings as they hit the market',
  'Salt therapy as practiced at The Salt Room Lake Country is a drug free treatment that not only brings relief but prevents reoccurrence over time.',
  'When your primary insurance falls short of covering losses for which you are liable, an umbrella policy can help take care of the difference.',
  'Please arrive at least 15 minutes early if you are using insurance to help cover the cost of the class.',
  'Rubberized TPU around the toe box for both medial and lateral traction on the mat so you can attack your opponent at any angle.',
  'Some traveling insurance coverage do not cover terrorism.',
  'The majority of journey traveling insurance policy will certainly cover journey termination, journey disruption or hold-ups, clinical, oral, emergency situation clinical transport, shed travel luggage, unintentional fatality, economic default of airline companies, and also cruise ship lines.',
  'Some traveling insurance coverage plans 

In [ ]:
aut_e_dataset.push_to_hub("an-eve/automatic-metaphors-entailment", private=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/an-eve/automatic-metaphors-entailment/commit/0914b995d1e145b28d6d7c3d2e12d157d764b9ee', commit_message='Upload dataset', commit_description='', oid='0914b995d1e145b28d6d7c3d2e12d157d764b9ee', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_aut_e_dataset = aut_e_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/644 [00:00<?, ? examples/s]

In [ ]:
args_test = TrainingArguments("Automatic-Metaphors",
         per_device_eval_batch_size=BATCH_SIZE,
         seed=128)

eval_trainer = Trainer(
    model=model,
    args=args_test,
    train_dataset=tokenized_aut_e_dataset,
    eval_dataset=tokenized_aut_e_dataset,
    compute_metrics=compute_metrics)

eval_trainer.evaluate()

{'eval_loss': 0.14239178597927094,
 'eval_accuracy': 0.9611801242236024,
 'eval_runtime': 21.5027,
 'eval_samples_per_second': 29.95,
 'eval_steps_per_second': 0.977}